# Assignment #1
## Greedy Motif Search

In [2]:
from Bio import SeqIO
import numpy as np
import math

In [ ]:
def totalcount(x, pseudocount):
    alpha = ['A', 'T', 'G', 'C']
    counts = np.array([np.sum(x==nt) for nt in alpha])
    finalcount = (counts + pseudocount)/(len(x) + 4*pseudocount)
    return finalcount

def profile(DNA, k, pos, log_odd=True):
    N = len(DNA)# or pos0
    x = np.empty((N, k), dtype=str) #matrix with k-mers
    W = np.empty((4, k)) #position weight matrix
    for i in range(N):
        x[i, :] = list(DNA[i][pos[i]:pos[i]+k])
    for j in range(k):
        W[:,j] = totalcount(x[:,j], pseudocount=1)
    if log_odd==True: # log-odds matrix
        W2 = np.log(W*4)/np.log(4)
    else: 
        W2 = W
    return W2

def llr(profile, s):
    ref = {
        'A':0,
        'T':1,
        'G':2,
        'C':3
    }
    llratio = 0
    for i in range(len(s)):
        llratio += profile[ref[s[i]], i]
    return llratio

def motifpos(profile, DNA):
    N = len(DNA)
    k = profile.shape[1]
    pos = np.empty(N, dtype=int)
    for i in range(N):
        maxllr = -math.inf
        for j in range(len(DNA[i])-k+1):
            llrval = llr(profile, DNA[i][j:j+k])
            if llrval > maxllr:
                maxllr = llrval
                pos[i] = j
    return pos   

In [ ]:
strong = [str(record.seq) for record in SeqIO.parse("synth_50_strong.fa", "fasta")]
weak = [str(record.seq) for record in SeqIO.parse("synth_50_weak.fa", "fasta")]


GreedyMotifSearch(weak, 6, 1000):
    best_score = -math.inf
    for i in range(len(weak[0]) - k + 1):
        pos = np.array([i])
        pwm = profile(weak, k, pos)
        score = sum([llr(pwm, weak[i][pos[i]:pos[i]+k]) for i in range(len(weak))])
        if score > best_score:
            best_score = score
            best_pos = pos
    return best_pos

k = 6 #Motif length
N = len(weak)
pos = np.array([np.random.randint(0, len(weak[i]) - k) for i in range(N)])

while True:
    old_pos = pos.copy()
    pwm = profile(weak, k, pos)
    pos = motifpos(pwm, weak)
    if np.array_equal(pos, old_pos):
        break

print("Final motif positions:", pos)
